In [1]:
import json
import glob
import numpy as np
import cv2
import copy

In [2]:
video_files =  glob.glob('D:/IIIT/IIIT_Shri_City/BTP/code/COMP90055_Research_Project-master/ResNextLSTM/Dataset/*.mp4')
#video_files1 =  glob.glob('/content/dfdc_train_part_0/*.mp4')
#video_files += video_files1
frame_count = []
for video_file in video_files:
  cap = cv2.VideoCapture(video_file)
  if(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))<150):
    video_files.remove(video_file)
    continue
  frame_count.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
print("frames" , frame_count)
print("Total number of videos: " , len(frame_count))
print('Average frame per video:',np.mean(frame_count))

frames [299, 299, 299, 299, 299, 299, 299, 299, 299, 299, 299, 300, 299, 300, 299, 299, 299, 299, 300, 300]
Total number of videos:  20
Average frame per video: 299.2


In [3]:
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [4]:
# # to extract frame
# def frame_extract(path):
#   vidObj = cv2.VideoCapture(path) 
#   success = 1
#   while success:
#       success, image = vidObj.read()
#       if success:
#           yield image

# import os
# # os.makedirs('F:/PYTHON10/deepfake2/Face_Only_Data')
# import face_recognition
# from tqdm.autonotebook import tqdm
# # process the frames
# def create_face_videos(path_list,out_dir):
#   already_present_count =  glob.glob(out_dir+'*.mp4')
#   print("No of videos already present " , len(already_present_count))
#   for path in tqdm(path_list):
#     out_path = os.path.join(out_dir,path.split('/')[-1])
#     file_exists = glob.glob(out_path)
#     if(len(file_exists) != 0):
#       print("File Already exists: " , out_path)
#       continue
#     frames = []
#     flag = 0
#     face_all = []
#     frames1 = []
#     out = cv2.VideoWriter(out_path,cv2.VideoWriter_fourcc('M','J','P','G'), 30, (112,112))
#     for idx,frame in enumerate(frame_extract(path)):
#       #if(idx % 3 == 0):
#       if(idx <= 150):
#         frames.append(frame)
#         if(len(frames) == 4):
#           faces = face_recognition.batch_face_locations(frames)
#           for i,face in enumerate(faces):
#             if(len(face) != 0):
#               top,right,bottom,left = face[0]
#             try:
#               out.write(cv2.resize(frames[i][top:bottom,left:right,:],(112,112)))
#             except:
#               pass
#           frames = []
#     try:
#       del top,right,bottom,left
#     except:
#       pass
#     out.release()

In [4]:
import cv2
import os
import glob
import face_recognition
from tqdm import tqdm


def frame_extract(path):
    vidObj = cv2.VideoCapture(path)
    success = True
    while success:
        success, image = vidObj.read()
        if success:
            yield image

# Function to create face videos
def create_face_videos(path_list, out_dir):
    already_present_count = len(glob.glob(os.path.join(out_dir, '*.mp4')))
    print("Number of videos already present:", already_present_count)
    
    for path in tqdm(path_list):
        out_path = os.path.join(out_dir, os.path.basename(path))
        out_path = out_path.split('.')[0] + ".mp4"  
        if os.path.exists(out_path):
            print("File already exists:", out_path)
            continue
        
        out = None
        frames_processed = 0
        
        for frame in frame_extract(path):
            if frames_processed > 150:  
                break
                
            face_locations = face_recognition.face_locations(frame)
            if face_locations:  # If faces are detected
                top, right, bottom, left = face_locations[0]
                face_image = frame[top:bottom, left:right]
                face_image_resized = cv2.resize(face_image, (112, 112))
                
                if out is None:
                    out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*'MP4V'), 30, (112, 112))
                
                out.write(face_image_resized)
                
            frames_processed += 1
            
        if out is not None:
            out.release()



In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [6]:
create_face_videos(video_files,'D:/IIIT/IIIT_Shri_City/BTP/code/COMP90055_Research_Project-master/ResNextLSTM/Face_Only_Data/')

Number of videos already present: 0


  0%|          | 0/20 [00:00<?, ?it/s]

100%|██████████| 20/20 [50:38<00:00, 151.93s/it]
